In [1]:
import argparse
import logging
import db.mariadb as mariadb
import os
import MySQLdb

import imageio
import numpy as np

import keras
from keras.preprocessing import image

from keras.applications.resnet50 import preprocess_input
from requests_futures.sessions import FuturesSession
from PIL.Image import DecompressionBombError

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(format='%(asctime)s-%(levelname)s-%(name)s - %(message)s')


model = keras.applications.resnet50.ResNet50(include_top=False, pooling='avg')


Using TensorFlow backend.
2019-03-18 18:36:08,324-WARNING-tensorflow - From /usr/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
# Change according to your database setup (these are the defaults, see .env file)

con = mariadb.get_connection("127.0.0.1", 3308, "image_processing", "user", "user_pw")

files_batch = mariadb.get_files_data(0, 800000, con)

con.close()

# Use every 100th image of the 800000
files_batch = files_batch[::100]
features_train = []
features_test = []

features_train_info = []
features_test_info = []

# Create training and test features
counter = 1
for (img_id, name, path, url) in files_batch:
    try:
        img = image.load_img(path, target_size=(224, 224))
        # print(type(img))
        img_data = image.img_to_array(img)
        # print(type(img_data))
        # print(img_data.shape)
        img_data = np.expand_dims(img_data, axis=0)
        # print(img_data.shape)
        img_data = preprocess_input(img_data)
        # print(img_data.shape)
        res_net_feature = model.predict(img_data)
        # print(res_net_feature.shape)
        res_net_feature = np.array(res_net_feature).flatten()
        # print(res_net_feature.shape)

        if counter % 3 == 0:
            features_test.append(res_net_feature)
            features_test_info.append((img_id, name, path, url))
        else:
            features_train.append(res_net_feature)
            features_train_info.append((img_id, name, path, url))

        counter += 1

    except OSError as e:
        print(e)
    except DecompressionBombError as e:
        print(e)
        print(path)

features_train = np.array(features_train)
features_test = np.array(features_test)

print(features_train.shape)
print(len(features_train_info))

print(features_test.shape)
print(len(features_test_info))

cannot identify image file '/mnt/idai_cloud/FADatenbankabb0700c/oA_14610.jpg'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0700b/FA-Kae54-22_002921808,02.jpg'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0700d/FA996-12_0002921601,11.jpg'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0700d/Mal1043-01_21751,03.jpg'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0700d/Mal18-07_140891,01.jpg'


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (131627440 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (96499176 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (129393132 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


cannot identify image file '/mnt/idai_cloud/FADatenbankabb0575/D-DAI-ROM-56.927_002922407.jpg'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0577/D-DAI-ROM-67.16_56977.jpg'
cannot identify image file '/mnt/idai_cloud/FADatenbankabb0577/D-DAI-ROM-71.624_000300141352.jpg'


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (100186268 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Image size (205393290 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.
/mnt/idai_cloud/dai-rom-fotothek-2007/Bestand-D-DAI-ROM-2007.5627.JPG


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (163745498 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (127422594 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0067.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0175.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0277.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0376.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0594.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0694.JPG'
cannot identify image file '/mnt/idai_cloud/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0794.JPG'
(5323, 2048)
5323
(2661, 2048)
2661


In [3]:
# We need to scale the features to values between 0 and 1, because the sigmoid layer (last autoencoder layer) 
# produces values in that range. Without scaling the original values down, the autoencoder won't be able to 
# reproduce input values > 1.

print(np.max(features_train))
print(np.max(features_test))

if(np.max(features_train) > np.max(features_test)):
    features_train_scaled = features_train / np.max(features_train)
    features_test_scaled = features_test / np.max(features_train)
else:
    features_train_scaled = features_train / np.max(features_test)
    features_test_scaled = features_test / np.max(features_test)

print(np.max(features_train_scaled))
print(np.max(features_test_scaled))

29.489676
21.073988
1.0
0.71462256


In [6]:
# Initialize and run autoencoder

from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam, Adadelta
from keras import regularizers
import keras.backend as K

def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

# Stop if there has been no improvement for 100 epochs
es = keras.callbacks.EarlyStopping(verbose=1, patience=100)
# Save best model while training
mc = keras.callbacks.ModelCheckpoint('best_model.h5')

callbacks_list = [es, mc]

m = Sequential()

m.add(Dense(512, activation='elu', input_shape=(features_train_scaled.shape[1],)))
m.add(Dense(32, activation='linear', name="bottleneck"))
m.add(Dense(512, activation='elu'))
m.add(Dense(features_train.shape[1], activation='sigmoid'))

m.compile(loss=euclidean_distance_loss, optimizer=Adadelta())

history = m.fit(features_train_scaled, features_train_scaled, batch_size=128, epochs=100000, verbose=1,
                validation_data=(features_test_scaled, features_test_scaled), callbacks=callbacks_list)


2019-03-18 20:45:21,289-WARNING-tensorflow - From /usr/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Train on 5323 samples, validate on 2661 samples
Epoch 1/100000
5323/5323 [==============================] - 4s 787us/step - loss: 5.4706 - val_loss: 1.1770
Epoch 2/100000
5323/5323 [==============================] - 3s 614us/step - loss: 1.1618 - val_loss: 1.1592
Epoch 3/100000
5323/5323 [==============================] - 3s 619us/step - loss: 1.1443 - val_loss: 1.1365
Epoch 4/100000
5323/5323 [==============================] - 3s 620us/step - loss: 1.1333 - val_loss: 1.1281
Epoch 5/100000
5323/5323 [==============================] - 3s 618us/step - loss: 1.1254 - val_loss: 1.1166
Epoch 6/100000
5323/5323 [==============================] - 3s 617us/step - loss: 1.1178 - val_loss: 1.1115
Epoch 7/100000
5323/5323 [==============================] - 3s 620us/step - loss: 1.1114 - val_loss: 1.1082
Epoch 8/100000
5323/5323 [==============================] - 3s 624us/step - loss: 1.1084 - val_loss: 1.1026
Epoch 9/100000
5323/5323 [==============================] - 3s 618us/step - loss: 1.1027

Epoch 76/100000
5323/5323 [==============================] - 3s 608us/step - loss: 0.8142 - val_loss: 0.8197
Epoch 77/100000
5323/5323 [==============================] - 3s 591us/step - loss: 0.8141 - val_loss: 0.8138
Epoch 78/100000
5323/5323 [==============================] - 3s 592us/step - loss: 0.8103 - val_loss: 0.8142
Epoch 79/100000
5323/5323 [==============================] - 3s 589us/step - loss: 0.8071 - val_loss: 0.8065
Epoch 80/100000
5323/5323 [==============================] - 3s 588us/step - loss: 0.8053 - val_loss: 0.8051
Epoch 81/100000
5323/5323 [==============================] - 3s 588us/step - loss: 0.8055 - val_loss: 0.8064
Epoch 82/100000
5323/5323 [==============================] - 3s 592us/step - loss: 0.8016 - val_loss: 0.8036
Epoch 83/100000
5323/5323 [==============================] - 3s 592us/step - loss: 0.7982 - val_loss: 0.7984
Epoch 84/100000
5323/5323 [==============================] - 3s 592us/step - loss: 0.7973 - val_loss: 0.7969
Epoch 85/100000
532

5323/5323 [==============================] - 3s 623us/step - loss: 0.6982 - val_loss: 0.7029
Epoch 151/100000
5323/5323 [==============================] - 3s 624us/step - loss: 0.6970 - val_loss: 0.7068
Epoch 152/100000
5323/5323 [==============================] - 3s 623us/step - loss: 0.6957 - val_loss: 0.7163
Epoch 153/100000
5323/5323 [==============================] - 3s 619us/step - loss: 0.6965 - val_loss: 0.7037
Epoch 154/100000
5323/5323 [==============================] - 3s 615us/step - loss: 0.6937 - val_loss: 0.7037
Epoch 155/100000
5323/5323 [==============================] - 3s 620us/step - loss: 0.6938 - val_loss: 0.7162
Epoch 156/100000
5323/5323 [==============================] - 3s 620us/step - loss: 0.6934 - val_loss: 0.7087
Epoch 157/100000
5323/5323 [==============================] - 3s 618us/step - loss: 0.6932 - val_loss: 0.7070
Epoch 158/100000
5323/5323 [==============================] - 3s 619us/step - loss: 0.6895 - val_loss: 0.7055
Epoch 159/100000
5323/5323 

5323/5323 [==============================] - 3s 623us/step - loss: 0.6373 - val_loss: 0.6566
Epoch 225/100000
5323/5323 [==============================] - 3s 619us/step - loss: 0.6372 - val_loss: 0.6610
Epoch 226/100000
5323/5323 [==============================] - 3s 622us/step - loss: 0.6356 - val_loss: 0.6583
Epoch 227/100000
5323/5323 [==============================] - 3s 620us/step - loss: 0.6366 - val_loss: 0.6586
Epoch 228/100000
5323/5323 [==============================] - 3s 620us/step - loss: 0.6336 - val_loss: 0.6580
Epoch 229/100000
5323/5323 [==============================] - 3s 617us/step - loss: 0.6347 - val_loss: 0.6618
Epoch 230/100000
5323/5323 [==============================] - 3s 621us/step - loss: 0.6348 - val_loss: 0.6598
Epoch 231/100000
5323/5323 [==============================] - 3s 619us/step - loss: 0.6319 - val_loss: 0.6642
Epoch 232/100000
5323/5323 [==============================] - 3s 621us/step - loss: 0.6338 - val_loss: 0.6535
Epoch 233/100000
5323/5323 

5323/5323 [==============================] - 3s 590us/step - loss: 0.5985 - val_loss: 0.6323
Epoch 299/100000
5323/5323 [==============================] - 3s 589us/step - loss: 0.5995 - val_loss: 0.6376
Epoch 300/100000
5323/5323 [==============================] - 3s 593us/step - loss: 0.5973 - val_loss: 0.6360
Epoch 301/100000
5323/5323 [==============================] - 3s 590us/step - loss: 0.5977 - val_loss: 0.6329
Epoch 302/100000
5323/5323 [==============================] - 3s 591us/step - loss: 0.5974 - val_loss: 0.6351
Epoch 303/100000
5323/5323 [==============================] - 3s 591us/step - loss: 0.5970 - val_loss: 0.6373
Epoch 304/100000
5323/5323 [==============================] - 3s 594us/step - loss: 0.5976 - val_loss: 0.6325
Epoch 305/100000
5323/5323 [==============================] - 3s 589us/step - loss: 0.5964 - val_loss: 0.6344
Epoch 306/100000
5323/5323 [==============================] - 3s 601us/step - loss: 0.5952 - val_loss: 0.6358
Epoch 307/100000
5323/5323 

5323/5323 [==============================] - 3s 591us/step - loss: 0.5753 - val_loss: 0.6238
Epoch 373/100000
5323/5323 [==============================] - 3s 587us/step - loss: 0.5752 - val_loss: 0.6213
Epoch 374/100000
5323/5323 [==============================] - 3s 590us/step - loss: 0.5754 - val_loss: 0.6255
Epoch 375/100000
5323/5323 [==============================] - 3s 590us/step - loss: 0.5735 - val_loss: 0.6223
Epoch 376/100000
5323/5323 [==============================] - 3s 587us/step - loss: 0.5743 - val_loss: 0.6269
Epoch 377/100000
5323/5323 [==============================] - 3s 589us/step - loss: 0.5737 - val_loss: 0.6293
Epoch 378/100000
5323/5323 [==============================] - 3s 592us/step - loss: 0.5734 - val_loss: 0.6263
Epoch 379/100000
5323/5323 [==============================] - 3s 590us/step - loss: 0.5737 - val_loss: 0.6220
Epoch 380/100000
5323/5323 [==============================] - 3s 619us/step - loss: 0.5733 - val_loss: 0.6245
Epoch 381/100000
5323/5323 

5323/5323 [==============================] - 3s 593us/step - loss: 0.5582 - val_loss: 0.6249
Epoch 447/100000
5323/5323 [==============================] - 3s 590us/step - loss: 0.5575 - val_loss: 0.6208
Epoch 448/100000
5323/5323 [==============================] - 3s 590us/step - loss: 0.5577 - val_loss: 0.6266
Epoch 449/100000
5323/5323 [==============================] - 3s 593us/step - loss: 0.5585 - val_loss: 0.6188
Epoch 450/100000
5323/5323 [==============================] - 3s 615us/step - loss: 0.5573 - val_loss: 0.6222
Epoch 451/100000
5323/5323 [==============================] - 3s 618us/step - loss: 0.5577 - val_loss: 0.6218
Epoch 452/100000
5323/5323 [==============================] - 3s 622us/step - loss: 0.5571 - val_loss: 0.6209
Epoch 453/100000
5323/5323 [==============================] - 3s 620us/step - loss: 0.5573 - val_loss: 0.6236
Epoch 454/100000
5323/5323 [==============================] - 3s 617us/step - loss: 0.5580 - val_loss: 0.6213
Epoch 455/100000
5323/5323 

5323/5323 [==============================] - 3s 591us/step - loss: 0.5462 - val_loss: 0.6219
Epoch 521/100000
5323/5323 [==============================] - 3s 591us/step - loss: 0.5460 - val_loss: 0.6194
Epoch 522/100000
5323/5323 [==============================] - 3s 588us/step - loss: 0.5454 - val_loss: 0.6193
Epoch 523/100000
5323/5323 [==============================] - 3s 592us/step - loss: 0.5447 - val_loss: 0.6236
Epoch 524/100000
5323/5323 [==============================] - 3s 591us/step - loss: 0.5448 - val_loss: 0.6219
Epoch 525/100000
5323/5323 [==============================] - 3s 590us/step - loss: 0.5455 - val_loss: 0.6188
Epoch 526/100000
5323/5323 [==============================] - 3s 592us/step - loss: 0.5443 - val_loss: 0.6225
Epoch 527/100000
5323/5323 [==============================] - 3s 591us/step - loss: 0.5440 - val_loss: 0.6206
Epoch 528/100000
5323/5323 [==============================] - 3s 589us/step - loss: 0.5451 - val_loss: 0.6199
Epoch 529/100000
5323/5323 

In [7]:
from keras.models import load_model
best_model = load_model('best_model.h5', custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
encoder = Model(best_model.input, best_model.get_layer('bottleneck').output)

enc = encoder.predict(features_train_scaled) # returns the encoded values (32 floats instead of 2048)


In [48]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=100)

result = neigh.fit(features_train_scaled)
neighbours = result.kneighbors()

In [49]:
result_enc = neigh.fit(enc)
neighbours_enc = result_enc.kneighbors()

In [50]:
enc_r = best_model.predict(features_train_scaled)      # reconstruction, 2048 -> 32 -> 2048

In [51]:
result_enc_r = neigh.fit(enc_r)
neighbours_enc_r = result_enc.kneighbors()

In [52]:
np.min(features_train_scaled)

0.0

In [53]:
np.min(features_test_scaled)

0.0

In [54]:
np.max(features_train_scaled)

1.0

In [55]:
np.max(features_test_scaled)

0.71462256

In [56]:
print(neighbours[1][0])
print(neighbours_enc_r[1][0])
print(neighbours_enc[1][0])

[1821 2854 2761 3123 3441 1823 3599 3762 2723 2719 2075 4224 3082 3125
 1499  100 3534 4518 4444 4012 3225 4301 2945 3115 4093 2909 3120 2985
 4195 5149 3694   65 3334 4113  713  857 2642 3678  544 3896 2625 1129
 4018 4987 2780 1827 3116 2161 2033 2729  280  848 3104 5098  383 2389
 1894 5024 2302   49 4804  882 1197 3689 2813   63 2944 2744 2736 2362
 2707  623 3338 3224 3026 5078 2703 4808 4268 3757 1300 5023 1901  640
 1168 5161 3034 5047 4148 3004 4550 2990 3020 2264 5031 3212 4094 2611
 4443 5084]
[2854 4093 1821 1499 4018 4518 3123 3115 3762 3599 2761 4012 2642 2723
 3896 3125  280 3225 3082 3127 5149 3441 2945  100 3694 3107 4224 2075
  623 2999 4987 2985 1823   93 2739 3026 4516 4305 2780  383 3678 1827
 3338  848 1129 2707 2033 4113 4444 5024 1583 2625 2944 4301 3808 2843
 1197 2703 1902 3757   65 2742 2719 3104 3087 2634 4804 1894 5042 4041
 2161 2729 5098 1277 3534 4195 5023 3935 2744 3763 2389 1312 3019 3120
  635  858 4148   37  887 2909  713   49  882 5209 5132 1495 5161

In [57]:
print(neighbours[1][1])
print(neighbours_enc_r[1][1])
print(neighbours_enc[1][1])

[ 721 2822 3091 4785 4879 2877 1107  208 4134 1022  150  173 5060 1102
  177 2669 3232 2255  185 1888 2554 4854  125 1088  166 2595  870 2495
  179  204  216 4914 4848 1111 2266 1971 5275 4865   72  195 1095  145
 3647 2749  388  188 4631 1365 1109 2496 2482 1092 2481 3876 1036 1530
 1420  157  149 4916  131 3924 4909 1412  903 3788  196 4866  184 3069
 5310 4122 3613 5259  187 1106 2825 1567 3312  435 4703 4876 1012 5277
 1050 4660 3838 2005 3346 1536 2202 3295 3622 1096 5169  199 1098 4856
 3234  154]
[ 721 2822 3091 4879 4785  150 1971  187  208 3232  154 1022 5060 4134
 1107  148 1092  204 1365 1088  185 1109 1111 2595 2669 1420 1102  177
  216 1090  173  145 4914  166 5275 2749 2266 4660 3647 3788 1888 2495
 2877 3828 1842  125 1567 1530 2885 2255  188 1096 4854 3473 4916   72
 3741  184 2482 2202 2496 3295 4865  435 2529 1095  131  870 4909  179
 1378 3069  220  903    2  198 2825  196 3747 1412 3711 3924 2481 3876
 4876 4122 2186  186 4923  158 5045 5169 3360 4703 4860 3613 1036

In [58]:
print(neighbours[1][2])
print(neighbours_enc_r[1][2])
print(neighbours_enc[1][2])

[ 184  204 1096  225 1090 1088 1085 1089 2822 3232 1107 1092 1098 3363
 2671  721 4660 3937 3346 4914 2495  216 5255 5275  173 1097 2529 5252
  166 1106 1530 2481 4122 4876 4854 4916 3613 2202 1365 2551 1109 2941
 1036 2554  145 1111  179 1176 2672 1888 2674 1412  735  188 4785 4105
  129 4840 1536 1463  150 2669 2660 3473 4051 1102 1094  128 2595 4609
 3058 5060 2255 2187  398 4879 4616 2496 2839 3741 2877 5254 5310 1971
 4856 3769 4768 4690 4860 3234 1028 2996 5291 5301 3788 3091 2692 3824
  195 1105]
[1090  204  184 1107  225 1088 1089 1096 1085 1092 5255 3232 2495  721
 1109  173 4660 4914 3363 2822  216 1098 2202 2529 1365 3346 2671 4105
 4916 3473 2481 1530 4923 5291  128 4616 2524 4785 1036 5275  188  148
 1412  166 1971 1028 5254 4860 4638 3937  179 4122 1102 4876 1097  150
 2206  129 2266 3360 2672 4051 4003 1094 1011 2749 2674 4690 2839 3295
 5252 2941 2669 4609 2551 2255 2893 1112 1044 5196 3741 4854 3484 1176
 4107 2966   72 1106 2595 2554 2530 3058 3319 2494 1105 2885 3234

In [59]:
enc.shape

(5323, 32)

In [60]:
features_train_scaled.shape

(5323, 2048)

In [61]:
encoder.save('encoder.h5')

In [62]:
import math


def compare_list_order(list_a, list_b):
    error = 0
    
    for a_index, a_content in enumerate(list_a):
        try:
            b_index = list_b.index(a_content)
            error += math.fabs(a_index - b_index)
        except ValueError:
            error += len(list_a)
    return error / len(list_a)


overall_error = 0

for image_idx, image_neighbours in enumerate(neighbours[1]):
    overall_error += compare_list_order(list(image_neighbours), list(neighbours_enc[1][image_idx]))      

print(overall_error / (len(neighbours[1])))
    

46.220988164568816
